In [79]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [5]:
!pip install xlrd

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -m pip install --upgrade pip' command.


In [8]:
df_measure = pd.read_excel('measure.xls')
display(df_measure)

,x1,x2,x3,x4,x5,x6,x7,y
0,94,80,70,77,99,85,79,83
1,91,82,76,91,79,96,93,87
2,86,82,72,92,99,78,92,86
3,88,79,77,75,80,78,82,80
4,93,82,77,75,79,76,89,82
...,...,...,...,...,...,...,...,...
94,81,79,79,88,82,91,83,83
95,87,80,77,92,78,88,91,85
96,87,80,73,81,80,75,80,79
97,92,79,75,87,94,80,79,84


In [23]:
X_array, y_array = list(), list()
for i, row in df_measure.iterrows():
    X = [v for v in row[:-1]]
    y = [row[-1]]
    X_array.append(X)
    y_array.append(y)
X_array = np.array(X_array)
y_array = np.array(y_array)
print(X_array.shape)
print(y_array.shape)

(99, 7)
(99, 1)


In [25]:
from sklearn.model_selection import train_test_split
train_test_split??

Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Source:   
def train_test_split(*arrays,
                     test_size=None,
                     train_size=None,
                     random_state=None,
                     shuffle=True,
                     stratify=None):
    """Split arrays or matrices into random train and test subsets

    Quick utility that wraps input validation and
    ``next(ShuffleSplit().split(X, y))`` and application to input data
    into a single call for splitting (and optionally subsampling) data in a
    oneliner.

    Read more in the :ref:`User Guide <cross_validation>`.

    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.

    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.7)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(29, 7)
(70, 7)
(29, 1)
(70, 1)


In [90]:
lr_clf = LinearRegression()
rr_clf = Ridge(alpha=1)
la_clf = Lasso(alpha=0.01)
elastic = ElasticNet(alpha=0.001, l1_ratio=0.5)

train_set = [X_train, y_train]
test_set = [X_test, y_test]

lr_clf.fit(*train_set)
rr_clf.fit(*train_set)
la_clf.fit(*train_set)
elastic.fit(*train_set)

accuracy_lr = lr_clf.score(*test_set)
accuracy_rr = rr_clf.score(*test_set)
accuracy_la = la_clf.score(*test_set)
accuracy_elastic = elastic.score(*test_set)

print(f'lr : {accuracy_lr}')
print(f'rr : {accuracy_rr}')
print(f'la : {accuracy_la}')
print(f'elastic : {accuracy_elastic}')

lr : 0.9921487495278429
rr : 0.9921527876446189
la : 0.9921237780629828
elastic : 0.9921477401692032


In [92]:
def equation(clf):
    print('\n', type(clf))
    if not isinstance(clf, (Lasso, ElasticNet)):
        coefs = [coef for coef in clf.coef_[0]]
    else:
        coefs = [coef for coef in clf.coef_]
    factors = list()
    for i, coef in enumerate(coefs):
        factors.append(f'{coef}x{i}')
    factors.append(str(clf.intercept_[0]))
    equation = 'y = {0}'.format(' + '.join(factors))
    print(equation)

equation(lr_clf)
equation(rr_clf)
equation(la_clf)
equation(elastic)


 <class 'sklearn.linear_model._base.LinearRegression'>
y = 0.1320438497742886x0 + 0.15354473830008575x1 + 0.10530765491435896x2 + 0.1400067956153316x3 + 0.14050608044831223x4 + 0.15400376725073484x5 + 0.16018993602188897x6 + 0.8595681861094846

 <class 'sklearn.linear_model._ridge.Ridge'>
y = 0.13184287060083175x0 + 0.1522974537775257x1 + 0.10492424981539916x2 + 0.13992633101465993x3 + 0.14048908001333787x4 + 0.15407459222039802x5 + 0.16037443614164898x6 + 0.9915257783336529

 <class 'sklearn.linear_model._coordinate_descent.Lasso'>
y = 0.131568724580413x0 + 0.15097895806906889x1 + 0.10409211230288569x2 + 0.13982004086380972x3 + 0.14045455375035898x4 + 0.15427097189180405x5 + 0.16071380272337707x6 + 1.1488324140731123

 <class 'sklearn.linear_model._coordinate_descent.ElasticNet'>
y = 0.13201522112558958x0 + 0.1533950920424653x1 + 0.105246085912793x2 + 0.13999374436759005x3 + 0.14050248015554273x4 + 0.15401589616482492x5 + 0.16022248218453367x6 + 0.876157426388815


In [93]:
from sklearn.metrics import mean_squared_error

lr_pred = lr_clf.predict(X_test)
rr_pred = rr_clf.predict(X_test)
la_pred = la_clf.predict(X_test).reshape(-1, 1)
elastic_pred = elastic.predict(X_test).reshape(-1, 1)
print(lr_pred.shape, rr_pred.shape, la_pred.shape, elastic_pred.shape)

print(mean_squared_error(lr_pred, y_test))
print(mean_squared_error(rr_pred, y_test))
print(mean_squared_error(la_pred, y_test))
print(mean_squared_error(elastic_pred, y_test))

(70, 1) (70, 1) (70, 1) (70, 1)
0.11837122038389329
0.11831033876859912
0.11874770853450589
0.11838643821631553


In [ ]:
# Ridge
# y = 0.13184287060083175x0 + 0.1522974537775257x1 + 0.10492424981539916x2 + 0.13992633101465993x3 + 0.14048908001333787x4 + 0.15407459222039802x5 + 0.16037443614164898x6 + 0.9915257783336529